In [4]:
import numpy as np
import pandas as pd
#import seaborn as sns
import matplotlib.pyplot as plt


In [5]:
Fin_data = pd.read_excel('./Fin_data_01.xlsx')
Fin_data

,Year,Month - name,Month -sequence,Date,Business Line,"Amount, $",Expense subgroup,Revenue / Expense Group,Revenue or expense
0,2023,January,1,2023-01-31,Nutrition and Food Supplements,153000,NaN,Sales,Revenue
1,2023,January,1,2023-01-31,Nutrition and Food Supplements,27000,NaN,Consulting and professional services,Revenue
2,2023,January,1,2023-01-31,Nutrition and Food Supplements,6000,NaN,Other income,Revenue
3,2023,January,1,2023-01-31,Nutrition and Food Supplements,-15000,Rent,Opex,Expense
4,2023,January,1,2023-01-31,Nutrition and Food Supplements,-9000,Equipment,Opex,Expense
...,...,...,...,...,...,...,...,...,...
575,2023,December,1,2023-12-31,Sportswear,-4000,Packaging,COGS,Expense
576,2023,December,1,2023-12-31,Sportswear,-7000,Shipping,COGS,Expense
577,2023,December,1,2023-12-31,Sportswear,-9000,Sales,COGS,Expense
578,2023,December,1,2023-12-31,Sportswear,-110000,Labor,COGS,Expense


In [6]:
Fin_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 580 entries, 0 to 579
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Year                     580 non-null    int64         
 1   Month - name             580 non-null    object        
 2   Month -sequence          580 non-null    int64         
 3   Date                     580 non-null    datetime64[ns]
 4   Business Line            580 non-null    object        
 5   Amount, $                580 non-null    int64         
 6   Expense subgroup         468 non-null    object        
 7   Revenue / Expense Group  580 non-null    object        
 8   Revenue or expense       580 non-null    object        
dtypes: datetime64[ns](1), int64(3), object(5)
memory usage: 40.9+ KB


In [7]:
Fin_data.describe()

,Year,Month -sequence,Date,"Amount, $"
count,580.0,580.0,580,580.000000
mean,2023.0,1.0,2023-07-15 01:49:14.482758656,7438.137931
min,2023.0,1.0,2023-01-31 00:00:00,-234000.000000
25%,2023.0,1.0,2023-03-31 00:00:00,-30000.000000
50%,2023.0,1.0,2023-06-30 00:00:00,-10800.000000
75%,2023.0,1.0,2023-09-30 00:00:00,-1800.000000
max,2023.0,1.0,2023-12-31 00:00:00,750000.000000
std,0.0,0.0,NaN,118699.587071


In [8]:
Fin_data.columns

Index(['Year', 'Month - name', 'Month -sequence', 'Date', 'Business Line',
       'Amount, $', 'Expense subgroup', 'Revenue / Expense Group',
       'Revenue or expense'],
      dtype='object')

In [9]:
#Now we shall rename columns

Fin_data.rename(columns= {'Month - name': 'Month_name'}, inplace= True)

In [10]:
Fin_data.rename(columns= {'Month -sequence' : 'Month_sequence', 'Amount, $': 'Amount', 'Expense subgroup' : 'Expense_subgroup', 'Revenue or expense' : 'Rev_or_exp'
                          , 'Revenue / Expense Group' : 'Rev_exp_group','Business Line': 'Biz_line' }, inplace= True)

In [11]:
#now we extract week from date 

def week_of_mon(col):
    first_day_of_mon = col.replace(day = 1)
    dom = col.day
    adjusted_dom = first_day_of_mon.weekday() + dom
    return int(np.ceil(adjusted_dom)/7)

Fin_data['week'] = Fin_data['Date'].apply(week_of_mon)



In [45]:
Fin_data['week'] = 'week' + ' ' + Fin_data['week'].astype(str)
Fin_data['Quarter'] = Fin_data['Date'].dt.quarter
Fin_data['Quarter'] = 'qtr ' + Fin_data['Quarter'].astype(str)
Fin_data['mon_no'] = Fin_data['Date'].dt.month

Fin_data

,Year,Month_name,Month_sequence,Date,Biz_line,Amount,Expense_subgroup,Rev_exp_group,Rev_or_exp,week,Quarter,mon_day,week_day,mon_no
0,2023,January,1,2023-01-31,Nutrition and Food Supplements,153000,NaN,Sales,Revenue,week week 5,qtr 1,31,Tuesday,1
1,2023,January,1,2023-01-31,Nutrition and Food Supplements,27000,NaN,Consulting and professional services,Revenue,week week 5,qtr 1,31,Tuesday,1
2,2023,January,1,2023-01-31,Nutrition and Food Supplements,6000,NaN,Other income,Revenue,week week 5,qtr 1,31,Tuesday,1
3,2023,January,1,2023-01-31,Nutrition and Food Supplements,-15000,Rent,Opex,Expense,week week 5,qtr 1,31,Tuesday,1
4,2023,January,1,2023-01-31,Nutrition and Food Supplements,-9000,Equipment,Opex,Expense,week week 5,qtr 1,31,Tuesday,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
575,2023,December,1,2023-12-31,Sportswear,-4000,Packaging,COGS,Expense,week week 5,qtr 4,31,Sunday,12
576,2023,December,1,2023-12-31,Sportswear,-7000,Shipping,COGS,Expense,week week 5,qtr 4,31,Sunday,12
577,2023,December,1,2023-12-31,Sportswear,-9000,Sales,COGS,Expense,week week 5,qtr 4,31,Sunday,12
578,2023,December,1,2023-12-31,Sportswear,-110000,Labor,COGS,Expense,week week 5,qtr 4,31,Sunday,12


In [44]:
Fin_data['mon_day'] = Fin_data['Date'].dt.day

Fin_data['week_day'] = Fin_data['Date'].dt.day_name()

In [15]:
Fin_data['Expense_subgroup'].value_counts()

Expense_subgroup
Other               72
Rent                36
Marketing           36
Equipment           36
Payroll             36
R&D                 36
Materials           36
Packaging           36
Shipping            36
Sales               36
Labor               36
Interest and tax    36
Name: count, dtype: int64

In [16]:
#here we pull data for expenses only

rev_only = Fin_data.query('Rev_or_exp == "Revenue"')
total_rev = rev_only['Amount'].sum()
total_rev

np.int64(17559820)

In [17]:
#now we look into total_expense

exp_only = Fin_data.query('Rev_or_exp == "Expense"')
exp_only
total_exp = exp_only['Amount'].sum()
total_exp

np.int64(-13245700)

In [18]:
Fin_data['Rev_exp_group'].value_counts()

Rev_exp_group
COGS                                    216
Opex                                    216
Other income                             40
Sales                                    36
Consulting and professional services     36
Interest and tax                         36
Name: count, dtype: int64

In [19]:
Fin_data['Rev_or_exp'].value_counts()

Rev_or_exp
Expense    468
Revenue    112
Name: count, dtype: int64

In [20]:
#Now we import our data dict
data_dict = pd.read_excel('./Fin_data_01.xlsx', sheet_name= 'dictionary')
data_dict

,Year,Year of revenue/expense
0,Month - name,Month of revenue/expense
1,Month -sequence,"Month of revenue/expense, expressed as number"
2,Date,"Date of revenue/expense, expressed as the last..."
3,Business Line,Business line generating revenue / expense (Sp...
4,"Amount, $",Revenue or expense amount in USD
5,Expense subgroup,Additional subgroups categorizing expenses ass...
6,Income / Expense Group,Subcategory of revenue / expense. Revenue subc...
7,Income or expense,Column indicating if associated amount is reve...
8,NaN,NaN
9,NaN,NaN


In [21]:
Fin_data.query('Rev_exp_group == "COGS"')

,Year,Month_name,Month_sequence,Date,Biz_line,Amount,Expense_subgroup,Rev_exp_group,Rev_or_exp,week,Quarter,mon_day,week_day
9,2023,January,1,2023-01-31,Nutrition and Food Supplements,-20000,Materials,COGS,Expense,week 5,qtr 1,31,Tuesday
10,2023,January,1,2023-01-31,Nutrition and Food Supplements,-1200,Packaging,COGS,Expense,week 5,qtr 1,31,Tuesday
11,2023,January,1,2023-01-31,Nutrition and Food Supplements,-2700,Shipping,COGS,Expense,week 5,qtr 1,31,Tuesday
12,2023,January,1,2023-01-31,Nutrition and Food Supplements,-3000,Sales,COGS,Expense,week 5,qtr 1,31,Tuesday
13,2023,January,1,2023-01-31,Nutrition and Food Supplements,-65000,Labor,COGS,Expense,week 5,qtr 1,31,Tuesday
...,...,...,...,...,...,...,...,...,...,...,...,...,...
575,2023,December,1,2023-12-31,Sportswear,-4000,Packaging,COGS,Expense,week 5,qtr 4,31,Sunday
576,2023,December,1,2023-12-31,Sportswear,-7000,Shipping,COGS,Expense,week 5,qtr 4,31,Sunday
577,2023,December,1,2023-12-31,Sportswear,-9000,Sales,COGS,Expense,week 5,qtr 4,31,Sunday
578,2023,December,1,2023-12-31,Sportswear,-110000,Labor,COGS,Expense,week 5,qtr 4,31,Sunday


In [22]:
Fin_data[['Biz_line', 'Expense_subgroup', 'Amount', 'Rev_exp_group']]

,Biz_line,Expense_subgroup,Amount,Rev_exp_group
0,Nutrition and Food Supplements,NaN,153000,Sales
1,Nutrition and Food Supplements,NaN,27000,Consulting and professional services
2,Nutrition and Food Supplements,NaN,6000,Other income
3,Nutrition and Food Supplements,Rent,-15000,Opex
4,Nutrition and Food Supplements,Equipment,-9000,Opex
...,...,...,...,...
575,Sportswear,Packaging,-4000,COGS
576,Sportswear,Shipping,-7000,COGS
577,Sportswear,Sales,-9000,COGS
578,Sportswear,Labor,-110000,COGS


In [23]:
#Fin_data.loc[Fin_data['Rev_exp_group'] == 'Consulting and professional services',['Biz_line', 'Expense_subgroup', 'Amount', 'Rev_exp_group']]

In [24]:
#Fin_data.loc[['Biz_line', 'Expense_subgroup', 'Amount', 'Rev_exp_group']]

Fin_data.loc[Fin_data['Rev_exp_group'] == 'COGS',['Biz_line', 'Expense_subgroup', 'Amount', 'Rev_exp_group']]

,Biz_line,Expense_subgroup,Amount,Rev_exp_group
9,Nutrition and Food Supplements,Materials,-20000,COGS
10,Nutrition and Food Supplements,Packaging,-1200,COGS
11,Nutrition and Food Supplements,Shipping,-2700,COGS
12,Nutrition and Food Supplements,Sales,-3000,COGS
13,Nutrition and Food Supplements,Labor,-65000,COGS
...,...,...,...,...
575,Sportswear,Packaging,-4000,COGS
576,Sportswear,Shipping,-7000,COGS
577,Sportswear,Sales,-9000,COGS
578,Sportswear,Labor,-110000,COGS


In [25]:
Fin_data.loc[Fin_data['Rev_exp_group'] == 'Opex',['Biz_line', 'Expense_subgroup', 'Amount', 'Rev_exp_group']]

,Biz_line,Expense_subgroup,Amount,Rev_exp_group
3,Nutrition and Food Supplements,Rent,-15000,Opex
4,Nutrition and Food Supplements,Equipment,-9000,Opex
5,Nutrition and Food Supplements,Marketing,-30000,Opex
6,Nutrition and Food Supplements,Payroll,-40000,Opex
7,Nutrition and Food Supplements,R&D,-30000,Opex
...,...,...,...,...
569,Sportswear,Equipment,-40000,Opex
570,Sportswear,Marketing,-40000,Opex
571,Sportswear,Payroll,-50000,Opex
572,Sportswear,R&D,-5000,Opex


Exploratory Data Analysis

In [26]:
#Total revenue, expense and net profit

df_rev_exp = ['tot_rev', 'tot_exp', 'net_profit']
df_rev_exp = pd.DataFrame(df_rev_exp)

#tot_rev =Fin_data[Fin_data['Rev_or_exp'] == 'Revenue', ['Amount'].sum()]
tot_rev_01 = Fin_data.groupby('Rev_or_exp')['Amount'].sum().reset_index(name= 'Amount')
tot_rev_01

,Rev_or_exp,Amount
0,Expense,-13245700
1,Revenue,17559820


In [27]:
#total profit, total expenses and net profit
tot_rev = Fin_data.loc[Fin_data['Rev_or_exp'] == 'Revenue', ['Amount']].sum()
tot_rev

tot_exp = Fin_data.loc[Fin_data['Rev_or_exp'] == 'Expense', 'Amount'].sum()
tot_exp

net_profit = tot_rev + tot_exp
net_profit

df_rev_exp =pd.DataFrame({'tot_rev': tot_rev,
              'tot_exp': tot_exp,
              'net_profit' : net_profit})
df_rev_exp

,tot_rev,tot_exp,net_profit
Amount,17559820,-13245700,4314120


Rev and expense across biz lines

In [28]:
#tot_revenue acros biz line

tot_rev_data_ext = Fin_data.loc[Fin_data['Rev_or_exp']== 'Revenue', ['Biz_line', 'Amount', 'Rev_exp_group', 'Rev_or_exp']]


pd.set_option('display.max_rows', 200)
tot_rev_biz_line = tot_rev_data_ext.groupby('Biz_line')['Amount'].sum().reset_index(name = 'tot_rev').sort_values('tot_rev', ascending = False)
tot_rev_biz_line

,Biz_line,tot_rev
1,Sports equipment,8908500
2,Sportswear,6807700
0,Nutrition and Food Supplements,1843620


In [29]:
df = Fin_data.loc[Fin_data['Rev_or_exp']== 'Revenue', ['Biz_line', 'Amount', 'Rev_exp_group', 'Rev_or_exp']]
df.head()

,Biz_line,Amount,Rev_exp_group,Rev_or_exp
0,Nutrition and Food Supplements,153000,Sales,Revenue
1,Nutrition and Food Supplements,27000,Consulting and professional services,Revenue
2,Nutrition and Food Supplements,6000,Other income,Revenue
16,Nutrition and Food Supplements,22500,Consulting and professional services,Revenue
17,Nutrition and Food Supplements,9000,Consulting and professional services,Revenue


In [30]:
#total_expense across biz lines

#first we extract the data needed for expenses

tot_exp_data_ext = Fin_data.loc[Fin_data['Rev_or_exp']=='Expense', ['Biz_line', 'Amount', 'Expense_subgroup','Rev_exp_group', 'Rev_or_exp']]


pd.set_option('display.max_rows', 470)
tot_exp_biz_line = tot_exp_data_ext.groupby('Biz_line')['Amount'].sum().reset_index(name= 'tot_exp').sort_values('tot_exp', ascending = False)
tot_exp_biz_line

,Biz_line,tot_exp
0,Nutrition and Food Supplements,-2556500
2,Sportswear,-4070900
1,Sports equipment,-6618300


#tot exp across various sub_groups

In [31]:
sub_group_tot_exp = tot_exp_data_ext.groupby('Expense_subgroup')['Amount'].sum().reset_index(name = 'tot_exp').sort_values('tot_exp', ascending = True)
sub_group_tot_exp

,Expense_subgroup,tot_exp
2,Labor,-4492000
7,Payroll,-1790000
0,Equipment,-1333000
4,Materials,-1065000
3,Marketing,-1055000
1,Interest and tax,-929400
9,Rent,-745000
8,R&D,-540000
10,Sales,-447000
5,Other,-405500


In [32]:
#total rev across various Rev_exp_group

tot_rev_group = tot_rev_data_ext.groupby('Rev_exp_group')['Amount'].sum().reset_index(name= 'tot_rev').sort_values('tot_rev', ascending= False)
tot_rev_group

,Rev_exp_group,tot_rev
2,Sales,13776630
0,Consulting and professional services,2817000
1,Other income,966190


In [33]:
#total expenses across various Rev_exp_group

tot_exp_group = tot_exp_data_ext.groupby('Rev_exp_group')['Amount'].sum().reset_index(name= 'tot_exp').sort_values('tot_exp', ascending = True)
tot_exp_group

,Rev_exp_group,tot_exp
0,COGS,-6711500
2,Opex,-5604800
1,Interest and tax,-929400


In [34]:
Fin_data.groupby('Rev_exp_group')['Amount'].sum().reset_index(name='total').sort_values('total', ascending= False)

,Rev_exp_group,total
5,Sales,13776630
1,Consulting and professional services,2817000
4,Other income,966190
2,Interest and tax,-929400
3,Opex,-5604800
0,COGS,-6711500


Overall trend analysis for biz line, expense_subgroup, rev_exp_subgroup and Rev_or_exp_group

Monthly Trends

In [48]:
#extracting required data for trend overview

trend_rev_extract = Fin_data.loc[Fin_data['Rev_or_exp'] == 'Revenue', ['Month_name', 'mon_no', 'Quarter', 'week', 'week_day', 'Biz_line', 'Rev_exp_group', 'Rev_or_exp', 'Amount' ]]

trend_exp_extract = Fin_data.loc[Fin_data['Rev_or_exp'] == 'Expense', ['Month_name', 'mon_no','Quarter', 'week', 'week_day', 'Biz_line', 'Rev_exp_group', 'Expense_subgroup', 'Rev_or_exp', 'Amount' ]]



In [50]:
#total rev by month

tot_rev_by_month = trend_rev_extract.groupby(['mon_no','Month_name'])['Amount'].sum().reset_index(name= 'tot_rev').sort_values('mon_no',ascending = True)
tot_rev_by_month

,mon_no,Month_name,tot_rev
0,1,January,1949000
1,2,February,1567700
2,3,March,1313350
3,4,April,1156100
4,5,May,1698500
5,6,June,1719200
6,7,July,1400050
7,8,August,1304850
8,9,September,1120890
9,10,October,1322475


In [51]:
#total exp by month
tot_exp_by_month = trend_exp_extract.groupby(['mon_no','Month_name'])['Amount'].sum().reset_index(name= 'tot_exp').sort_values('mon_no', ascending = True)
tot_exp_by_month


,mon_no,Month_name,tot_exp
0,1,January,-1254500
1,2,February,-1089000
2,3,March,-1010800
3,4,April,-963400
4,5,May,-1230900
5,6,June,-1226100
6,7,July,-1111200
7,8,August,-1036300
8,9,September,-1029300
9,10,October,-1027400


In [52]:
#month over month change in revenue

tot_rev_by_month['mom_diff'] = tot_rev_by_month['tot_rev'] - tot_rev_by_month['tot_rev'].shift(1)
tot_rev_by_month['mom_%_change'] = tot_rev_by_month['tot_rev'].pct_change().round(3) *100
tot_rev_by_month.fillna(0, inplace= True)
tot_rev_by_month


,mon_no,Month_name,tot_rev,mom_diff,mom_%_change
0,1,January,1949000,0.0,0.0
1,2,February,1567700,-381300.0,-19.6
2,3,March,1313350,-254350.0,-16.2
3,4,April,1156100,-157250.0,-12.0
4,5,May,1698500,542400.0,46.9
5,6,June,1719200,20700.0,1.2
6,7,July,1400050,-319150.0,-18.6
7,8,August,1304850,-95200.0,-6.8
8,9,September,1120890,-183960.0,-14.1
9,10,October,1322475,201585.0,18.0


In [53]:
#mom diff and percentage change in expenses

tot_exp_by_month.head()

tot_exp_by_month['mom_diff'] = tot_exp_by_month['tot_exp'] - tot_exp_by_month['tot_exp'].shift(1)
tot_exp_by_month['mom_%_change'] = tot_exp_by_month['tot_exp'].pct_change().round(3)*100
tot_exp_by_month.fillna(0, inplace= True)
tot_exp_by_month


,mon_no,Month_name,tot_exp,mom_diff,mom_%_change
0,1,January,-1254500,0.0,0.0
1,2,February,-1089000,165500.0,-13.2
2,3,March,-1010800,78200.0,-7.2
3,4,April,-963400,47400.0,-4.7
4,5,May,-1230900,-267500.0,27.8
5,6,June,-1226100,4800.0,-0.4
6,7,July,-1111200,114900.0,-9.4
7,8,August,-1036300,74900.0,-6.7
8,9,September,-1029300,7000.0,-0.7
9,10,October,-1027400,1900.0,-0.2


Now we look at quarterly analysis

In [60]:
#rev by quarter
trend_rev_by_qtr = trend_rev_extract.groupby('Quarter')['Amount'].sum().reset_index(name='tot_rev').sort_values('Quarter', ascending= True)
trend_rev_by_qtr['mom_diff'] = trend_rev_by_qtr['tot_rev'] - trend_rev_by_qtr['tot_rev'].shift(1)
trend_rev_by_qtr['mom_%_diff'] = trend_rev_by_qtr['tot_rev'].pct_change().round(3)*100
trend_rev_by_qtr.fillna(0, inplace = True)

trend_rev_by_qtr

,Quarter,tot_rev,mom_diff,mom_%_diff
0,qtr 1,4830050,0.0,0.0
1,qtr 2,4573800,-256250.0,-5.3
2,qtr 3,3825790,-748010.0,-16.4
3,qtr 4,4330180,504390.0,13.2


In [65]:
#change in expense by quarter
trend_exp_by_qtr = trend_exp_extract.groupby('Quarter')['Amount'].sum().reset_index(name='tot_exp').sort_values('Quarter', ascending= True)
trend_exp_by_qtr['mom_diff'] = trend_exp_by_qtr['tot_exp'] - trend_exp_by_qtr['tot_exp'].shift(1)
trend_exp_by_qtr['mom_%_diff'] = trend_exp_by_qtr['tot_exp'].pct_change().round(3)*100
trend_exp_by_qtr.fillna(0, inplace = True)

trend_exp_by_qtr

,Quarter,tot_exp,mom_diff,mom_%_diff
0,qtr 1,-3354300,0.0,0.0
1,qtr 2,-3420400,-66100.0,2.0
2,qtr 3,-3176800,243600.0,-7.1
3,qtr 4,-3294200,-117400.0,3.7


Now we shall look into Performance for each business lines and carry out a monthly and quarterly comparison


Net profit by business line and sub groups

In [81]:
trend_exp_extract.head()

#expense by business line, exp group and subgroup
#Fin_data.groupby(['Month_name','Biz_line'])['Amount'].sum().reset_index(name= 'tot').sort_values(['Month_name', 'tot'], ascending= [True, False])
exp_by_biz_sub_grp = trend_exp_extract.groupby([ 'mon_no','Biz_line','Month_name'])['Amount'].sum().reset_index(name = 'tot_exp').sort_values(['Biz_line','mon_no', 'tot_exp'], ascending= [True, True, False])
exp_by_biz_sub_grp

,mon_no,Biz_line,Month_name,tot_exp
0,1,Nutrition and Food Supplements,January,-231500
3,2,Nutrition and Food Supplements,February,-230900
6,3,Nutrition and Food Supplements,March,-212600
9,4,Nutrition and Food Supplements,April,-213800
12,5,Nutrition and Food Supplements,May,-233900
15,6,Nutrition and Food Supplements,June,-233300
18,7,Nutrition and Food Supplements,July,-205900
21,8,Nutrition and Food Supplements,August,-200900
24,9,Nutrition and Food Supplements,September,-192000
27,10,Nutrition and Food Supplements,October,-192300


In [82]:
exp_by_exp_grp = trend_exp_extract.groupby([ 'mon_no','Month_name', 'Rev_exp_group'])['Amount'].sum().reset_index(name = 'tot_exp').sort_values(['Rev_exp_group','mon_no', 'tot_exp'], ascending= [True, True, False])
exp_by_exp_grp

,mon_no,Month_name,Rev_exp_group,tot_exp
0,1,January,COGS,-662600
3,2,February,COGS,-507000
6,3,March,COGS,-513500
9,4,April,COGS,-482900
12,5,May,COGS,-641900
15,6,June,COGS,-634600
18,7,July,COGS,-580900
21,8,August,COGS,-546900
24,9,September,COGS,-535900
27,10,October,COGS,-515900


In [83]:
exp_by_exp_sub_grp = trend_exp_extract.groupby([ 'mon_no','Expense_subgroup','Month_name'])['Amount'].sum().reset_index(name = 'tot_exp').sort_values(['Expense_subgroup','mon_no', 'tot_exp'], ascending= [True, True, False])
exp_by_exp_sub_grp

,mon_no,Expense_subgroup,Month_name,tot_exp
0,1,Equipment,January,-93000
12,2,Equipment,February,-93000
24,3,Equipment,March,-93000
36,4,Equipment,April,-93000
48,5,Equipment,May,-117800
60,6,Equipment,June,-117800
72,7,Equipment,July,-117800
84,8,Equipment,August,-117800
96,9,Equipment,September,-117800
108,10,Equipment,October,-124000
